In [1]:
import json 
import os
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import pandas as pd
import re
import unicodedata
import itertools
import random
from model import EncoderRNN,LuongAttnDecoderRNN,Copy_seq2seq
from decoder import GreedySearchDecoder,BeamSearchDecoder
from train import Trainer
from corpus import Corpus
import config



In [2]:
import json 
import os
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import pandas as pd
import re
import unicodedata
import itertools
import random
from model import EncoderRNN,LuongAttnDecoderRNN,Copy_seq2seq
from decoder import GreedySearchDecoder,BeamSearchDecoder
from train import Trainer
from corpus import Corpus
import config


dataCor = Corpus(config.conv_path,config.wiki_path,config.save_dir)
dataCor.load()

trainData = dataCor.create_batches(config.batch_size,'train')
validData = dataCor.create_batches(config.batch_size,'valid')
testData = dataCor.create_batches(config.batch_size,'test')


Loading prepared data from ../data/pointer/data_2.pt ...
Number of examples: TRAIN-37876 VALID-2383 TEST-7134
Loading prepared vocab from ../data/pointer/vocab2.pt ...
Vocabulary size: 12614


In [3]:
model = Copy_seq2seq(vocab_size=dataCor.vocab.num_words,
                     embedding_size=config.embedding_size,
                     hidden_size=config.hidden_size,
                     encoder_num_layers=config.encoder_num_layers,
                     decoder_num_layers=config.decoder_num_layers,
                     dropout=config.dropout)
if config.use_cuda:
    model.cuda(config.device)


# Optimizer definition
optimizer = getattr(torch.optim, config.optimizer)(
    model.parameters(), lr=config.lr)


# Run training iterations
print("Starting Training!")
save_dir = os.path.join(config.save_dir,'model')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

generator = GreedySearchDecoder(model)

print("Training starts ...")
trainer = Trainer(model=model, optimizer=optimizer, trainData=trainData,
                  validData=validData,vocab=dataCor.vocab,generator=generator,num_epochs=config.num_epochs,
                  save_dir=save_dir, log_steps=config.log_steps,
                  valid_steps=config.valid_steps, grad_clip=config.grad_clip)
if config.ckpt is not None:
    trainer.load(file_prefix=config.ckpt)
trainer.train()

result_file = os.path.join(config.save_dir,'greedy.txt')
evaluate_generation(generator=generator,
                   data_iter=testData,
                   vocab=dataCor.vocab,
                   save_file=result_file)

result_file = os.path.join(config.save_dir,'beamSearch.txt')
generator = BeamSearchDecoder(model)
evaluate_generation(generator=generator,
                   data_iter=testData,
                   vocab=dataCor.vocab,
                   save_file=result_file)

Starting Training!
Training starts ...


/media/data/wangan/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Valid Loss:  7.819047657605842

================================== Model Training ===================================

[Train][ 1][200/592]   Average Loss: 5.72427392945501   TIME-0.92
[Train][ 1][400/592]   Average Loss: 5.505270378973396   TIME-0.75

--------------------------------- Model Evaulation ---------------------------------
[Valid][ 1][400/592]   Valid Loss: 5.43173445646488
Saved model state to '../data/pointer/model/state_epoch_1.model'
Saved train state to '../data/pointer/model/state_epoch_1.train'
Saved best model state to '../data/pointer/model/best.model' with new best valid loss 5.432
-------------------------------------------------------------------------------------

Generation starts ...
Saved generation results to '../data/pointer/model/valid_1.result'
Avg_Len-6.950   Bleu-0.1866/0.1462   Inter_Dist-0.0101/0.0353
Target:   AVG_LEN-10.935   Inter_Dist-0.0972/0.4934
Saved model state to '../data/pointer/model/state_epoch_1.model'
Saved train state to '../data/poi

[Train][ 8][200/592]   Average Loss: 4.521262462921499   TIME-0.72
[Train][ 8][400/592]   Average Loss: 4.679757197404049   TIME-0.93

--------------------------------- Model Evaulation ---------------------------------
[Valid][ 8][400/592]   Valid Loss: 4.483111611846484
Saved model state to '../data/pointer/model/state_epoch_8.model'
Saved train state to '../data/pointer/model/state_epoch_8.train'
Saved best model state to '../data/pointer/model/best.model' with new best valid loss 4.483
-------------------------------------------------------------------------------------

Generation starts ...
Saved generation results to '../data/pointer/model/valid_8.result'
Avg_Len-8.513   Bleu-0.2543/0.2018   Inter_Dist-0.0027/0.0077
Target:   AVG_LEN-10.935   Inter_Dist-0.0972/0.4934
Saved model state to '../data/pointer/model/state_epoch_8.model'
Saved train state to '../data/pointer/model/state_epoch_8.train'

================================== Model Training ==================================

[Train][15][200/592]   Average Loss: 3.9022433661797953   TIME-0.75
[Train][15][400/592]   Average Loss: 4.40640249656655   TIME-0.77

--------------------------------- Model Evaulation ---------------------------------
[Valid][15][400/592]   Valid Loss: 4.238357072435276
Saved model state to '../data/pointer/model/state_epoch_15.model'
Saved train state to '../data/pointer/model/state_epoch_15.train'
Saved best model state to '../data/pointer/model/best.model' with new best valid loss 4.238
-------------------------------------------------------------------------------------

Generation starts ...
Saved generation results to '../data/pointer/model/valid_15.result'
Avg_Len-8.098   Bleu-0.2388/0.1883   Inter_Dist-0.0054/0.0135
Target:   AVG_LEN-10.935   Inter_Dist-0.0972/0.4934
Saved model state to '../data/pointer/model/state_epoch_15.model'
Saved train state to '../data/pointer/model/state_epoch_15.train'

================================== Model Training =============================

...

Saved generation results to '../data/pointer/model/valid_73.result'
Avg_Len-7.759   Bleu-0.2345/0.1846   Inter_Dist-0.0272/0.0924
Target:   AVG_LEN-10.935   Inter_Dist-0.0972/0.4934
Saved model state to '../data/pointer/model/state_epoch_73.model'
Saved train state to '../data/pointer/model/state_epoch_73.train'

================================== Model Training ===================================

[Train][74][200/592]   Average Loss: 3.1539688480093275   TIME-0.76
[Train][74][400/592]   Average Loss: 3.1630101902213803   TIME-0.72

--------------------------------- Model Evaulation ---------------------------------
[Valid][74][400/592]   Valid Loss: 3.786163357556236
-------------------------------------------------------------------------------------

Generation starts ...
Saved generation results to '../data/pointer/model/valid_74.result'
Avg_Len-7.767   Bleu-0.2461/0.1938   Inter_Dist-0.0277/0.0925
Target:   AVG_LEN-10.935   Inter_Dist-0.0972/0.4934
Saved model state to '../data/po

[Train][82][200/592]   Average Loss: 3.115457254745127   TIME-0.69
[Train][82][400/592]   Average Loss: 2.8521392565464305   TIME-0.77

--------------------------------- Model Evaulation ---------------------------------
[Valid][82][400/592]   Valid Loss: 3.775152609848529
-------------------------------------------------------------------------------------

Generation starts ...
Saved generation results to '../data/pointer/model/valid_82.result'
Avg_Len-7.829   Bleu-0.2403/0.1895   Inter_Dist-0.0306/0.1066
Target:   AVG_LEN-10.935   Inter_Dist-0.0972/0.4934
Saved model state to '../data/pointer/model/state_epoch_82.model'
Saved train state to '../data/pointer/model/state_epoch_82.train'

================================== Model Training ===================================

[Train][83][200/592]   Average Loss: 3.1401658820128797   TIME-0.70
[Train][83][400/592]   Average Loss: 2.984609791876721   TIME-0.72

--------------------------------- Model Evaulation ----------------------------

Saved generation results to '../data/pointer/model/valid_90.result'
Avg_Len-7.637   Bleu-0.2338/0.1848   Inter_Dist-0.0318/0.1120
Target:   AVG_LEN-10.935   Inter_Dist-0.0972/0.4934
Saved model state to '../data/pointer/model/state_epoch_90.model'
Saved train state to '../data/pointer/model/state_epoch_90.train'

================================== Model Training ===================================

[Train][91][200/592]   Average Loss: 2.7658853329286353   TIME-0.74
[Train][91][400/592]   Average Loss: 2.9994908945604424   TIME-0.77

--------------------------------- Model Evaulation ---------------------------------
[Valid][91][400/592]   Valid Loss: 3.777973012066541
-------------------------------------------------------------------------------------

Generation starts ...
Saved generation results to '../data/pointer/model/valid_91.result'
Avg_Len-8.556   Bleu-0.2551/0.2017   Inter_Dist-0.0323/0.1151
Target:   AVG_LEN-10.935   Inter_Dist-0.0972/0.4934
Saved model state to '../data/po

[Train][99][200/592]   Average Loss: 2.7436219125417383   TIME-0.78
[Train][99][400/592]   Average Loss: 2.8969245370191232   TIME-0.80

--------------------------------- Model Evaulation ---------------------------------
[Valid][99][400/592]   Valid Loss: 3.7757377689739133
-------------------------------------------------------------------------------------

Generation starts ...
Saved generation results to '../data/pointer/model/valid_99.result'
Avg_Len-8.542   Bleu-0.2507/0.1982   Inter_Dist-0.0345/0.1238
Target:   AVG_LEN-10.935   Inter_Dist-0.0972/0.4934
Saved model state to '../data/pointer/model/state_epoch_99.model'
Saved train state to '../data/pointer/model/state_epoch_99.train'

================================== Model Training ===================================

[Train][100][200/592]   Average Loss: 2.9692218729948934   TIME-0.71
[Train][100][400/592]   Average Loss: 2.9084511172494025   TIME-0.78

--------------------------------- Model Evaulation -----------------------

NameError: name 'evaluate_generation' is not defined